# Classification d'images Pure-Python à partir de zéro (pour débutants)

Ce classificateur d'images montre à quel point il est facile de créer un classificateur d'images fonctionnel en utilisant seulement du python de base et des bibliothèques faciles à utiliser et bien documentées. Il est également créé pour comprendre les concepts de base du traitement d'images.

## Objectifs

Nous voulons créer un script python simple qui reconnaît un numéro manuscrit. L'image d'entrée doit avoir une taille fixe de 64x64 pixels, les images sont en couleurs RGB. Ecrivons un script python qui analyse l'image et nous donne une prédiction du nombre qui y est écrit. Nous prendrons les mesures suivantes pour atteindre cet objectif :

* Tout d'abord, nous avons un <u>ensemble d'images</u> qui affichent tous les chiffres possibles que notre script doit reconnaître. Cet ensemble d'images servira de données d'apprentissage pour notre scénario.


* Ensuite, nous voulons écrire une méthode qui ouvre et <u>normalise</u> notre image pour que seule l'information pertinente soit conservée. La couleur et les autres bruits gênants doivent être complètement éliminés.


* Puis, créons une base de données contenant toutes les images d'apprentissage bien définies. Nous comparerons une nouvelle image d'entrée aux images de la base de données pour faire une estimation du nombre qu'elle affiche.


* Enfin, nous écrivons l'algorithme de comparaison. Nous prenons une nouvelle image d'entrée, la normalisons et la comparons à chaque image de notre base de données qui calcule la similarité en fonction des valeurs des pixels. La catégorie d'images ayant la plus grande similitude nous servira d'estimation.


Nous voulons baser notre algorithme sur l'apprentissage, ce qui signifie que nous ne faisons pas du hardcode pour lui dire comment il reconnaîtra le chiffre "1", mais qu'il le reconnaîtra par lui-même. Nous le faisons en lui fournissant un ensemble d'images d'entraînement bien définies, donc des images qui sont déjà classées, nommées d'après le chiffre qu'elles affichent. Notre algorithme lira les images et la classe à laquelle elles appartiennent, et les stockera dans une base de données. Lorsqu'une nouvelle image inconnue est fournie, l'algorithme fait une boucle dans la base de données et calcule la similitude avec les images connues. Décomposons-les en plus petits morceaux et expliquons-les en détail.

## Les bibliothèques nécessaires

Ok, maintenant que nous avons une idée de ce que notre script doit faire, nous allons les traduire en code python réel.

Pour ouvrir des images, nous utiliserons la classe Image de **PIL** (Python Image Library). Pour effectuer efficacement les calculs sur l'image, nous utiliserons la bibliothèque mathématique scientifique **numpy**. Enfin, pour stocker notre base de données d'images et l'ouvrir plus tard, nous utiliserons **pickle**. Alors que pickle est une bibliothèque standard, les deux autres doivent être installés à l'aide de `pip`.

Importons ces bibliothèques maintenant. 

In [ ]:
from PIL import Image 
import numpy as np 
import pickle

## La structure de la classe

Nous voulons que notre classificateur d'images soit une classe. Cela a plusieurs avantages, par exemple, cela permet de l'importer facilement dans un autre projet par la suite, et cela nous donne un projet bien structuré dans le style OOP.

Notre classe doit avoir un **constructeur** et une méthode **str**, ainsi qu'une méthode pour **créer** & **ouvrir** une base de données d'images, **normaliser** une image, normaliser une base entière et **classifier** une image. 

Le `constructor` doit prendre un `databaseName` avec lequel nous classons l'image. La méthode `createDatabase` prend un *dossier* dans lequel sont stockées les images, et un *nom d'une base de données* qui sera créée contenant les représentations de ces images. La méthode `normalize` prend une `image` qu'il faut normaliser. La méthode `normalizeDatabase` prend une `normalizationFunction`, juste au cas où nous implémenterions une fonction de normalisation différente plus tard (alors nous pouvons passer celle que nous voulons appliquer sur la BDD comme argument dans la méthode. Si vous n'êtes pas familier avec ce concept, lisez cet article [ici](https://dbader.org/blog/python-first-class-functions).Plutôt direct. Notre schéma de classes vides ressemblera à ceci :

In [ ]:
class ImageClassifier:
    def __init__(self, database):
        pass
        
    def __str__(self):
        pass
        
    @staticmethod
    def createDatabase(imageFolder, databasename):
        pass
        
    def openDatabase(self):
        pass
        
    @staticmethod
    def normalizeBinary(image):
        pass
        
    @staticmethod
    def normalizeNot(image):
        pass
        
    def normalizeDatabase(self, normFunction):
        pass
        
    def classifyImage(self, img, normFunction):
        pass

Ne vous inquiétez pas du *@staticmethod*, nous y reviendrons un peu plus tard. 

## Constructor et méthode str

Commençons d'abord par les choses faciles. Un classificateur d'images fonctionne toujours avec une base de données, notre constructeur doit donc définir une base de données vide et le nom de la base de données. La méthode string retourne une description simple de la classe, donc de la base de données avec laquelle le classificateur courant fonctionne.

In [ ]:
class ImageClassifier:
    def __init__(self, databaseName):
        self._database = None
        self._databaseName = databaseName

    def __str__(self):
        if not self._database == None:
            return ("ImageRegognizer with database of" + str(len(self._database)) + " different Classes, each containing " + str(len(self._database[0])) + " images.")

    @staticmethod
    def createDatabase(imageFolder, databaseName):
        pass

    def openDatabase(self):
        pass

    @staticmethod
    def normalizeBinary(image):
        pass

    @staticmethod
    def normalizeNot(image):
        pass

    def normalizeDatabase(self, normFunction):
        pass
    
    def classifyImage(self, img, normFunction):
        pass

Ok, c'était facile. Continuons avec la création d'une telle base de données.

## Créer la base de données

Nous avons dit qu'un classificateur utilise une base de données pour classifier les images. Mais la création d'une telle base de données est en quelque sorte indépendante du classificateur lui-même. 

La création d'une base de données appartient au `ImageClassifier` en général, mais pas à une instance particulière de la classe. C'est pourquoi nous avons utilisé l'attribut *@staticmethod* : Il définit que nous pouvons utiliser cette méthode sans avoir une instance de la classe. Ceci est visible par l'absence de l'argument *self* à la première position : la Méthode ne sait pas à quelle instance elle appartient. En fait, il est complètement découplé de N'IMPORTE QUELLE instance et se trouve juste dans la classe elle-même. Mais qu'est-ce que ça veut dire ? Il exprime le fait que la méthode ci-dessous n'utilise pas de champs ou de méthodes spécifiques à une instance et peut être appelée librement sans instance. C'est super cool parce que nous pouvons alors créer une nouvelle base de données simplement en appelant
`ImageClassifier.createDatabase(...)`, et ensuite utiliser cette base de données pour n'importe quel classificateur particulier que nous voulons.

Bien, nous utilisons une *@staticmethod* alors, mais que fait exactement `.createDatabase()` maintenant ? Qu'est-ce qu'on veut faire ? Nous voulons enregistrer une base de données sur le disque contenant toutes les images sous forme de tableaux. Pourquoi des tableaux ? Parce que nous manipulerons les images plus tard, ce que nous ne pouvons faire que si nous les voyons comme des tableaux. Nous devons donc ouvrir nos images, les convertir en tableaux et les enregistrer dans une base de données. Passons à travers cette étape par étape. 

### Le schéma d'images

L'utilisateur nous fournira le nom du dossier dans lequel se trouvent les images. En tant que développeur, nous définissons comment il doit nommer les images pour que nous puissions identifier leur classe (<u>étiqueter les images</u>). Il doit suivre la structure suivante :

image_folder<br>
&emsp;&emsp;|-+ digit_index.jpg

Dans le dossier image_folder, toutes les images sont localisées et nommées par le chiffre qu'elles affichent et un index incrémentiel. Un exemple rapide ici :

image_folder/<br>
├── 0_0.jpg<br>
├── 0_1.jpg<br>
├── 0_2.jpg<br>
├── 0_3.jpg<br>
.<br>
.<br>
.<br>
├── 1_0.jpg<br>
├── 1_1.jpg<br>
├── 1_2.jpg<br>
├── 1_3.jpg<br>
.<br>
.<br>
.<br>

Pour entraîner initialement votre classificateur, vous devrez créer de telles images par vous-même. Faites attention à toujours dessiner le chiffre un peu différent pour que notre classifieur puisse classifier les chiffres mal dessinés également. À continuation l'ensemble utilisé dans ce tuto:

![Numéro 1](tuto_images/test_images.png)

Nous voulons faire simple ici et stocker les images dans un dictionnaire où les clés sont les chiffres et les valeurs sont des listes contenant la représentation-tableau de toutes les images correspondantes. 

````Pyton
number_db = {}
for i in range(10) :
    number_db[i] = []
```

Faisons maintenant une boucle dans notre dossier d'images et ouvrons les images par leur nom bien défini : nous supposons ici que l'utilisateur fournit toujours 16 images d'une sorte, donc 16 images affichant un zéro, 16 affichant un 1 et ainsi de suite. Nous ouvrons l'image par cette convention de nommage, la convertissons en tableau et l'insérons dans la liste correspondante dans la base de données des dictées :

```Python
for number in range(len(number_db.keys())):
    for index in range(16): # Change 16 to the number of images you have per category
        image = Image.open(imageFolder + "/" + str(number) + "_" + str(index) + ".jpg")
        number_db[number].append(np.array(image).tolist())
```

En appelant `np.array(image).tolist()`, nous convertissons notre objet Image `PIL` en une liste python. La liste représente l'image, chaque sous-liste de la liste représente une ligne, et chaque sous-liste de CETTE liste représente un pixel, tandis que chaque pixel comprend trois valeurs ; R(ed), G(reen), B(lue). L'image suivante devrait montrer clairement à quoi ressemble la conversion de liste.

![Image à lister](tuto_images/image_theory_1.png)

Maintenant que nous avons un dict qui nous sert de base de données, sauvegardons ce dict sur le disque en utilisant pickle. 

```Python
with open(databaseName + ".pkl", "wb") as db:
    pickle.dump(number_db,db)
```

Dans la fonction principale de notre script, nous pouvons maintenant appeler cette méthode et créer un nouveau pickle de base de données.

```Python
if __name__ == "__main__":
    ImageClassifier.createDatabase("images", "number_db")
```

Ceci crée un nouveau *number_db.pkl* dans notre dossier principal. 

In [ ]:
class ImageClassifier:
    def __init__(self, databaseName):
        self._database = None
        self._databaseName = databaseName

    def __str__(self):
        if not self._database == None:
            return ("ImageRegognizer with database of" + str(len(self._database)) + " different Classes, each containing " + str(len(self._database[0])) + " images.")

    @staticmethod
    def createDatabase(imageFolder, databaseName):
        number_db = {}
        for i in range(10):
            number_db[i] = []
        for number in range(len(number_db.keys())):
            for index in range(16):
                image = Image.open(imageFolder + "/" + str(number) + "_" + str(index) + ".jpg")
                number_db[number].append(np.array(image).tolist())

        with open(databaseName + ".pkl", "wb") as db:
            pickle.dump(number_db,db)

    def openDatabase(self):
        pass

    @staticmethod
    def normalizeBinary(image):
        pass

    @staticmethod
    def normalizeNot(image):
        pass

    def normalizeDatabase(self, normFunction):
        pass
    
    def classifyImage(self, img, normFunction):
        pass

## Ouvrir la base de données

Ok, maintenant que nous avons créé notre base de données, nous avons besoin d'un moyen de dire à notre classificateur de l'utiliser. De cette manière, nous créons une méthode `openDatabase`. Pourquoi n'avons-nous pas à spécifier le nom de la base de données ? Rappelez-vous que, lorsque nous créons un nouveau Classifier, nous devons spécifier quelle base de données utiliser, de sorte que le nom de la base de données est déjà une variable d'instance de la class-instance elle-même. 

```Python
def openDatabase(self):
    with open(self._databaseName+".pkl", "rb") as db:
        self._database = pickle.load(db)
```

Le code est simple : Ouvrez la base de données et chargez le contenu de la base de données depuis le pickle dans la variable self._database. 

In [ ]:
class ImageClassifier:
    def __init__(self, databaseName):
        self._database = None
        self._databaseName = databaseName

    def __str__(self):
        if not self._database == None:
            return ("ImageRegognizer with database of" + str(len(self._database)) + " different Classes, each containing " + str(len(self._database[0])) + " images.")

    @staticmethod
    def createDatabase(imageFolder, databaseName):
        number_db = {}
        for i in range(10):
            number_db[i] = []
        for number in range(len(number_db.keys())):
            for index in range(16):
                image = Image.open(imageFolder + "/" + str(number) + "_" + str(index) + ".jpg")
                number_db[number].append(np.array(image).tolist())

        with open(databaseName + ".pkl", "wb") as db:
            pickle.dump(number_db,db)

    def openDatabase(self):
        with open(self._databaseName+".pkl", "rb") as db:
            self._database = pickle.load(db)

    @staticmethod
    def normalizeBinary(image):
        pass

    @staticmethod
    def normalizeNot(image):
        pass

    def normalizeDatabase(self, normFunction):
        pass
    
    def classifyImage(self, img, normFunction):
        pass

## Normaliser l'image

Nous avons parlé un peu de normalisation, mais pas en détail. Pourquoi devrions-nous normaliser une image, et qu'est-ce que cela signifie ? Eh bien, la seule chose que nous avons définie au sujet de l'image d'entrée est qu'elle doit avoir une taille fixe de 64 par 64 pixels. Mais cela laisse encore une certaine marge de variation : qu'en est-il de la couleur du chiffre et de l'arrière-plan ? Peut-on comparer un neuf bleu sur fond vert ? Qu'en est-il d'une image numérisée, où l'image a un faible contraste et un fond gris ? Cela semble difficile à gérer, mais en fait, il est facile de réduire ces variables. 

Voyons à quoi peut ressembler une image d'entrée. 

![Noisy](tuto_images/test8_noisy.jpg)

Bien qu'elle ressemble au chiffre 8 de nos images d'essai (en fait, c'est la même image avec un certain style visuel), il est clair que nous ne pouvons pas comparer directement cette image à notre base de données et espérons obtenir un résultat d'estimation. Ce serait beaucoup mieux si chaque image avait une forme similaire.

![but bruyant](tuto_images/test8_noisy_goal.jpg)

Regardons une sélection de cette image et voyons comment nous pourrions atteindre cette réduction. Nous prenons une photo rapprochée de la région marquée en rouge (regardez de près, c'est juste une colonne de pixels et à peine visible).

![sélection](tuto_images/test8_noisy_selcetion.jpg)

C'est la même région, mais elle est maintenant agrandie. On peut voir certaines valeurs de pixels, le pixel jaunâtre le plus à gauche a la valeur RGB[244, 240, 125], le pixel le plus à droite a la valeur RGB[63, 74, 157]. Rappelez-vous que la partie jaunâtre n'appartient pas au chiffre, tandis que la partie bleue à droite appartient au chiffre. 

![colonne](tuto_images/reduction_1.jpg)

Maintenant, nous aimerions modifier ce tableau de pixels pour être en mesure de distinguer clairement quelle partie appartient au chiffre, et quelle partie n'appartient pas. Dans un premier temps, nous nous débarrassons des couleurs, parce que nous ne sommes en fait pas du tout intéressés par celles-ci. Pourquoi pas ? Parce que nous pouvons supposer que le chiffre est écrit sur quelque chose comme du papier, de sorte que le chiffre aura une couleur plus foncée que l'arrière-plan, peu importe de quelle couleur il s'agit. Nous voulons simplement examiner le contraste. Convertissons donc cette image en valeurs de Niveaux de gris uniquement en faisant la moyenne de chaque pixel : [244, 240, 125] devient 244 + 240 + 125 = 206, donc[206, 206, 206, 206].

![niveaux de gris](tuto_images/reduction_2.jpg)

Maintenant, allons un peu plus loin et faisons en sorte que chaque pixel qui est sombre soit entièrement noir, et chaque pixel qui est lumineux soit entièrement blanc. Nous faisons une coupe claire : Chaque pixel qui est plus clair que 255/2 (255 est la valeur la plus claire qu'un pixel peut avoir, 0 le plus foncé) comme clair, chaque pixel qui est plus foncé que 255/2 comme foncé. Nous obtenons le résultat suivant :

![bw](tuto_images/reduction_3.jpg)

On a presque fini ! Tu vois ces valeurs en pixels ? Ils sont toujours entièrement blancs[255, 255, 255, 255] ou entièrement noirs[0, 0, 0, 0]. Nous stockons donc des informations redondantes. Remplaçons[255, 255, 255, 255] par 1 et[0, 0, 0, 0] par 0.

![01](tuto_images/reduction_4.jpg)

Nous avons maintenant réussi à convertir non seulement une image colorée en niveaux de gris, mais aussi une image en niveaux de gris en binaire. 

![binarydemo](tuto_images/binary_demo.jpg)

### Le code

Ok, réalisons qu'en python. Tout d'abord, nous bouclons chaque colonne de l'image. Ensuite, on fait une boucle à travers tous les pixels de la colonne. Ensuite, nous bouclons à nouveau à travers chaque valeur de pixel et additionnons leurs valeurs de couleur. Si la valeur totale de la couleur divisée par trois est supérieure à 255/2, on attribue le pixel a 1, sinon 0, et enfin on retourne l'image.

```Python
@staticmethod
def normalizeBinary(image):
    for column in image:
        for pixel in range(len(column)):
            total_color = 0
            for color in column[pixel]:
                total_color += color
            if total_color/3 < 255/2:
                column[pixel] = 0
            else:
                column[pixel] = 1
    return image
```

## Normaliser l'ensemble de la base de données

Maintenant que nous pouvons normaliser une image, nous pouvons aussi normaliser une base de données en la bouclant simplement à travers chaque image qu'elle contient et en remplaçant les images par leurs normalisées.

```Python
def normalizeDatabase(self, normFunction):
    for number in range(len(self._database.keys())):
        for image in self._database[number]:
    image = normFunction(image)
```

## Le classificateur 

Maintenant que tant de travail de prétraitement est fait, le classificateur lui-même est assez simple. Ce que nous voulons faire, c'est ouvrir l'image, la convertir en tableau numpy, la normaliser avec la même fonction de normalisation que la base de données, puis la comparer à toutes les images de la base de données tout en gardant trace de la similarité entre chacune d'elles. 

Commençons par ouvrir et convertir les images :

```Python
def classifyImage(self, img, normFunction):
    test_image = Image.open(img)
    test_image = np.array(test_image).tolist()
    test_image = normFunction(test_image)
```

La partie de comparaison commence. Tout d'abord, nous initialisons un dict dans lequel nous allons stocker la similarité entre l'image fournie et chaque classe d'image dans la base de données. Ensuite, nous faisons une boucle dans la base de données et comparons les deux images, pixel pour pixel. Si les deux pixels correspondent, nous ajoutons deux points de confiance à notre dictionnaire de confiance pour la classe à laquelle nous comparons. Lorsque les pixels ne correspondent pas, on soustrait un point de confiance de la classe comparée. Donc notre dicton de confiance ressemble à ceci :


**Confiance_dict = {"0" : 55, "1":4, "2":32, "3":120    ...    , "8":432, "9":43}**


Il nous dit essentiellement combien de pixels correspondent par rapport à toutes les images de cette classe. Supposons que l'image d'entrée affiche le chiffre "8". En le comparant à toutes les images de la base de données qui affichent un "0", on peut obtenir environ 55 points de confiance. Cela semble beaucoup, mais la valeur absolue du nombre n'a pas d'importance, parce que lorsque nous comparons l'image d'entrée aux images de la base de données affichant également le chiffre "8", nous aurons (espérons-le) un nombre de confiance beaucoup plus élevé, comme 432 ! 

```Python
for i in range(10):
        confidence_dict[i] = 0
    for number in range(len(self._database.keys())):
        for image in self._database[number]:
            for column in range(len(image)):
                for pixel in range(len(image[column])):
                    if image[column][pixel] == test_image[column][pixel] == 1:
                        confidence_dict[number] += 2
                    if image[column][pixel] == test_image[column][pixel] == 0:
                        continue
    confidence_dict[number] -= 1
```

Attendez ! Pourquoi ajoute-t-on 2 points de confiance pour un pixel correspondant, mais soustrayons-nous seulement 1 pour un dématch ? Bonne question ! Supposons que vous traitez le chiffre "1" comme une image d'entrée et que vous le comparez à un autre "1" dans la base de données. Supposons que les images correspondent presque parfaitement, sauf qu'un pixel est un peu déplacé. Tu vois comment ça génère 2 points de confiance négatifs ? Ainsi, lorsqu'un pixel ne correspond pas, il est traité beaucoup moins bien que lorsqu'il correspond. Pour équilibrer le tout, nous ajoutons non pas un, mais deux points de confiance pour chaque match. 

![confidence-match](tuto_images/match_demo.jpg)

Maintenant, nous avons besoin d'un peu de statistiques pour obtenir une valeur en pourcentage de notre tableau de confiance. Tout d'abord, nous trions les confidences pour découvrir ce qu'est notre prédiction : le chiffre ayant le plus haut indice de confiance. Mais nous n'avons pas encore fini, nous voulons aussi indiquer à quel point nous sommes sûrs de notre prédiction. Si nous prédisons que l'image est un "1", mais que nous ne sommes sûrs qu'à 51% et qu'elle pourrait aussi être un "7", notre prédiction ne vaut pas grand chose. 


Pour être juste, nous devrions enfin montrer à quel point nous sommes certains. Pour le faire, nous comparons la certitude de notre prédiction avec la certitude de notre deuxième meilleure estimation, lorsque la deuxième estimation est presque aussi bonne que notre prédiction, nous sommes comme certains à 50 %. Mais si notre hypothèse est que la prédiction a un degré de confiance 3 fois plus élevé que notre seconde estimation, nous pouvons être sûrs à 100% de pouvoir vraiment faire face au nombre que nous avons prédit. 

Nous divisons 1 par notre meilleure estimation et multiplions le résultat par notre deuxième meilleure estimation, *100 pour obtenir une valeur en pourcentage. Arrondi à deux chiffres, nous obtenons une valeur en pourcentage propre et agréable.

```Python
max_confidence = sorted(confidence_dict.values(), reverse=True)
    for key, value in confidence_dict.items():
        if value == max_confidence[0]:
            return key, min(round(((1.0/max_confidence[0])*max_confidence[1])*100,2), 100.00)
```

Mettons tout ensemble :

In [ ]:
class ImageClassifier:
    def __init__(self, databaseName):
        self._database = None
        self._databaseName = databaseName

    def __str__(self):
        if not self._database == None:
            return ("ImageRegognizer with database of " + str(len(self._database)) + 
                    " different Classes, each containing " + str(len(self._database[0])) + " images.")

    @staticmethod
    def createDatabase(imageFolder, databaseName):
        number_db = {}
        for i in range(10):
            number_db[i] = []
        for number in range(len(number_db.keys())):
            for index in range(16):
                image = Image.open(imageFolder + "/" + str(number) + "_" + str(index) + ".jpg")
                number_db[number].append(np.array(image).tolist())

        with open(databaseName + ".pkl", "wb") as db:
            pickle.dump(number_db,db)

    def openDatabase(self):
        with open(self._databaseName+".pkl", "rb") as db:
            self._database = pickle.load(db)

    @staticmethod
    def normalizeBinary(image):
        for column in image:
            for pixel in range(len(column)):
                total_color = 0
                for color in column[pixel]:
                    total_color += color
                if total_color/3 < 255/2:
                    column[pixel] = 0
                else:
                    column[pixel] = 1
        return image

    @staticmethod
    def normalizeNot(image):
        pass

    def normalizeDatabase(self, normFunction):
        for number in range(len(self._database.keys())):
            for image in self._database[number]:
                image = normFunction(image)

    def classifyImage(self, img, normFunction):
        test_image = Image.open(img)
        test_image = np.array(test_image).tolist()
        test_image = normFunction(test_image)
        confidence_dict = {}
        for i in range(10):
            confidence_dict[i] = 0
        for number in range(len(self._database.keys())):
            for image in self._database[number]:
                for column in range(len(image)):
                    for pixel in range(len(image[column])):
                        if image[column][pixel] == test_image[column][pixel] == 1:
                            confidence_dict[number] += 2
                        if image[column][pixel] == test_image[column][pixel] == 0:
                            continue
                        confidence_dict[number] -= 1
        max_confidence = sorted(confidence_dict.values(), reverse=True)
        for key, value in confidence_dict.items():
            if value == max_confidence[0]:
                return key, min(round(((1.0/max_confidence[0])*max_confidence[1])*100,2), 100.00)

## Tester notre script

On a enfin fini ! Testons les performances de notre script. Ecrire une fonction principale qui est exécutée lorsque notre script est directement appelé et tester le script avec quelques images de test (assurez-vous de ne pas les inclure dans les images de formation, ce serait de la triche). 

J'ai ajouté quelques images de test appelées test_*digit*.jpg dans le dossier principal, chacune affichant un chiffre entre 0 et 9. Faisons un essai : Créez une nouvelle base de données avec nos images de formation, créez un nouveau classificateur avec cette base de données et ouvrez la base de données. Normalisez la base de données avec notre fonction `normalizeBinary`, puis bouclez à travers chacune des images test et appelez `.classifyImage()` avec la même fonction de normalisation. 

In [ ]:
#Use for testing ImageRecognizer Class
ImageClassifier.createDatabase("images", "number_db")

imageClassifier = ImageClassifier("number_db")

imageClassifier.openDatabase()

print(imageClassifier)

imageClassifier.normalizeDatabase(ImageClassifier.normalizeBinary)

for number in range(10):
    print(imageClassifier.classifyImage("test" + str(number) +".jpg", ImageClassifier.normalizeBinary))

## Mais attends !

Et si nous ne voulons pas normaliser les images, mais comparer les images brutes ? Nous devons fournir une fonction de normalisation, c'est obligatoire dans certains appels de fonction ! C'est vrai, nous devons trouver un moyen de contourner cela. Mais c'est facile, trichons un peu et créons une fonction de normalisation qui ne fait - rien. Renvoyez simplement l'image vierge à nouveau.

```Python
@staticmethod
    def normalizeNot(image):
        return image
```

## Améliorations

Ce script n'est que la base de la classification des images, n'est-ce pas ? Nous pourrions obtenir beaucoup plus de cet algorithme avec quelques ajustements. Par exemple, notre algorithme échoue immédiatement lorsqu'il compare un "1" qui se trouve dans la moitié gauche de l'image à un "1" qui se trouve sur la partie droite, car il ne trouve aucun pixel correspondant. Nous pourrions résoudre ce problème en coupant tous les espaces blancs en haut et à gauche du chiffre. De plus, nos images doivent être de taille fixe, donc nous pourrions essayer de les rendre un peu plus flexibles en les mettant d'abord à l'échelle. Ou nous pourrions rendre l'algorithme plus efficace en ne sauvegardant pas toutes nos images dans la base de données, mais en les additionnant en une seule image "moyenne totale" du chiffre, ce qui nous ferait gagner beaucoup de temps sur de plus grands ensembles de formation. Mais ces améliorations dépendent de vous ! 